In [1]:
!pip install ipywidgets
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to c:\users\pc\appdata\local\temp\pip-req-build-zirxz7gc
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached torch-1.10.1-cp37-cp37m-win_amd64.whl (226.6 MB)
  Using cached numpy-1.16.6-cp37-cp37m-win_amd64.whl (11.9 MB)
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached torch-1.10.0-cp37-cp37m-win_amd64.whl (226.6 MB)
INFO: pip is looking at multiple versions of sentencepiece to determine which version is compatible with other requirements. This could take a while.
  Using cached sentencepiece-0.1.96-cp37-cp37m-win_amd64.whl (1.1 MB)
INFO: pip is looking at multiple versions of mxnet to determine which version is compatible with other requ

  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' 'C:\Users\PC\AppData\Local\Temp\pip-req-build-zirxz7gc'
ERROR: Cannot install kobert because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [2]:
!git clone https://github.com/SKTBrain/KoBERT.git
%cd KoBERT
!pip install -r requirements.txt

C:\Users\PC\VSC\bert\KoBERT


fatal: destination path 'KoBERT' already exists and is not an empty directory.


  Using cached torch-1.10.1-cp37-cp37m-win_amd64.whl (226.6 MB)
  Using cached numpy-1.16.6-cp37-cp37m-win_amd64.whl (11.9 MB)
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached torch-1.10.0-cp37-cp37m-win_amd64.whl (226.6 MB)
INFO: pip is looking at multiple versions of sentencepiece to determine which version is compatible with other requirements. This could take a while.
  Using cached sentencepiece-0.1.96-cp37-cp37m-win_amd64.whl (1.1 MB)
INFO: pip is looking at multiple versions of mxnet to determine which version is compatible with other requirements. This could take a while.
  Using cached mxnet-1.7.0.post2-py2.py3-none-win_amd64.whl (33.1 MB)
  Using cached mxnet-1.7.0.post1-py2.py3-none-win_amd64.whl (33.0 MB)

ERROR: Cannot install -r requirements.txt (line 2), -r requirements.txt (line 3) and -r requirements.txt (line 4) because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [4]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
device = torch.device("cuda:0")

In [7]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. C:\Users\PC\VSC\bert\KoBERT\.cache\kobert_v1.zip
using cached model. C:\Users\PC\VSC\bert\KoBERT\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. C:\Users\PC\VSC\bert\KoBERT\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [11]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [12]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [13]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [14]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [15]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [16]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [17]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [18]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [19]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [20]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [21]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7051867842674255 train acc 0.578125
epoch 1 batch id 201 loss 0.5161126852035522 train acc 0.5763370646766169
epoch 1 batch id 401 loss 0.4631495773792267 train acc 0.6867206982543641
epoch 1 batch id 601 loss 0.4288637638092041 train acc 0.73681884359401
epoch 1 batch id 801 loss 0.40208566188812256 train acc 0.7654299313358303
epoch 1 batch id 1001 loss 0.32055723667144775 train acc 0.7820772977022977
epoch 1 batch id 1201 loss 0.3071320354938507 train acc 0.7947283513738551
epoch 1 batch id 1401 loss 0.36454036831855774 train acc 0.8040796752319772
epoch 1 batch id 1601 loss 0.32095015048980713 train acc 0.8116118831980013
epoch 1 batch id 1801 loss 0.27967509627342224 train acc 0.8179223348139922
epoch 1 batch id 2001 loss 0.26608070731163025 train acc 0.8235569715142429
epoch 1 batch id 2201 loss 0.2806864380836487 train acc 0.8281675942753294
epoch 1 train acc 0.8315246373720137


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 1 test acc 0.8827725383631714


  0%|          | 0/2344 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8975983056265985


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.4718216359615326 train acc 0.875
epoch 5 batch id 201 loss 0.036575496196746826 train acc 0.9738028606965174
epoch 5 batch id 401 loss 0.05199874937534332 train acc 0.9753740648379052
epoch 5 batch id 601 loss 0.13261984288692474 train acc 0.9757955490848585
epoch 5 batch id 801 loss 0.20332427322864532 train acc 0.9762796504369539
epoch 5 batch id 1001 loss 0.041516054421663284 train acc 0.977022977022977
epoch 5 batch id 1201 loss 0.015168735757470131 train acc 0.977583784346378
epoch 5 batch id 1401 loss 0.04586707800626755 train acc 0.9776275874375446
epoch 5 batch id 1601 loss 0.040245287120342255 train acc 0.9777873204247346
epoch 5 batch id 1801 loss 0.00872697215527296 train acc 0.9780764158800667
epoch 5 batch id 2001 loss 0.04306286200881004 train acc 0.9784873188405797
epoch 5 batch id 2201 loss 0.10302364826202393 train acc 0.978546683325761
epoch 5 train acc 0.9786556100682594


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 5 test acc 0.8977781329923273


In [24]:
import pandas as pd

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
  commnetslist = [] # 텍스트 데이터를 담을 리스트
  emo_list = [] # 감성 값을 담을 리스트
  for c in comment: # 모든 댓글
    commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정
    
  pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
  pdData = pdData.values
  test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=0)
  
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    # 이때, out이 예측 결과 리스트
    out = model(token_ids, valid_length, segment_ids)
	
    # e는 2가지 실수 값으로 구성된 리스트
    # 0번 인덱스가 더 크면 부정, 긍정은 반대
    for e in out:
      if e[0]>e[1]: # 부정
        value = 0
      else: #긍정
        value = 1
      emo_list.append(value)

  return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

In [30]:
comment = ['완전 좋아',
'너무하네','최고야']
getSentimentValue(comment, tok, max_len, batch_size, device)

[1, 0, 1]

In [23]:
!pip install pandas

  Using cached pandas-1.3.5-cp37-cp37m-win_amd64.whl (10.0 MB)
  Using cached pytz-2022.2.1-py2.py3-none-any.whl (500 kB)
